# **[Problem 1] Confirmation of competition contents**

* **What to learn and what to predic?**

This prompt us to learn about the **Transaction Information** relative to the ***clients*** and to be able to Predict the **Payment Abilities**  of the ***clients***. This is relative to the probability for the TARGET variable

* **What kind of file to create and submit to Kaggle?**

For each **SK_ID_CURR** in the test set, you must be able to predict a probability for the **TARGET** variable. 

**NOTE:** The file should contain a header and have the following format: **SK_ID_CURR,TARGET**
* 100001,0.1
* 100005,0.9
* 100013,0.2 etc.

* **What kind of index value will be used to evaluate the submissions?**

The submissions are evaluated on area under the **Receiver Operating Characteristic(ROC) curve** between the predicted probability and the observed target.

# **[Problem 2] Learning and verification**

* We import all needed libraries
* Load the Data (application_train.csv)
* Delete the Null data(empty data)
* Extract or separate them into variables

In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score


df = pd.read_csv('application_train.csv')


cleaned_df = df.dropna()


X = cleaned_df.loc[:,["AMT_INCOME_TOTAL","AMT_CREDIT","AMT_ANNUITY"]]
y = cleaned_df['TARGET']

In [6]:
df.head()#loading the data

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1.0,0.0,1.0,1.0,0.0,Laborers,1.0,2.0,2.0,WEDNESDAY,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1.0,0.0,1.0,1.0,0.0,Core staff,2.0,1.0,1.0,MONDAY,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1.0,1.0,1.0,1.0,0.0,Laborers,1.0,2.0,2.0,MONDAY,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1.0,0.0,1.0,0.0,0.0,Laborers,2.0,2.0,2.0,WEDNESDAY,17.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1.0,0.0,1.0,0.0,0.0,Core staff,1.0,2.0,2.0,THURSDAY,11.0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**We are spliting the data into traning and testing data (sklearn)**

* Standardizing the data
* fit and predict

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_train_trans = scaler.transform(X_train)
X_test_trans = scaler.transform(X_test)

reg = LinearRegression().fit(X_train_trans, y_train)

reg_pred = reg.predict(X_test_trans)

print("MSE:", mean_squared_error(y_true=y_test, y_pred=reg_pred))
print("ROC", roc_auc_score(y_test,reg_pred))

MSE: 0.0567142335416132
ROC 0.5245983935742973


**MSE** is very low which is a good indication

## **[Problem 3] Estimation on test data**

* Performing the estimation on the test data (application_test.csv ) and submit it to Kaggle.

In [ ]:
test_df.head()

In [ ]:
test_df = pd.read_csv('application_test.csv')

test_cleaned_df = test_df.dropna(axis=0)

test_X = test_cleaned_df.loc[:,["AMT_INCOME_TOTAL","AMT_CREDIT","AMT_ANNUITY"]]

test_scaler = StandardScaler()
test_X_test_trans = scaler.fit_transform(test_X)

test_reg_pred = reg.predict(test_X_test_trans)
#submitting to kaggle
kgl_submission = pd.concat([test_df['SK_ID_CURR'], pd.Series(test_reg_pred, name='TARGET')], axis=1)
kgl_submission = kgl_submission.fillna(0)
kgl_submission.at[648,'TARGET']= 0
kgl_submission.shape
kgl_submission.to_csv('kggl_submission.csv', index=False)

## **[Problem 4] Feature Engineering**

To improve accuracy, perform Feature Engineering from the following perspectives.
* Which feature to use?
* How to preprocess?

**Pattern 1 of training and validation.**

In [ ]:
# imputation
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

imp_mean = SimpleImputer(strategy='mean')

# deleting the missing values
imp_X = imp_mean.fit_transform(X)

# One hot encoding
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc_imp_X = enc.fit_transform(imp_X).toarray()

# splitting the data into training and testing data using train_test_split from sklearn
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(enc_imp_X, y, test_size=0.25, random_state=42)

# we standardized the data
scaler = StandardScaler()
scaler.fit(X_train_1)
X_train_trans_1 = scaler.transform(X_train_1)
X_test_trans_1 = scaler.transform(X_test_1)

# fitting the data
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=5)
lgb = lgbm.fit(X_train_trans_1, y_train_1)

# predicting
reg_pred_1 = lgb.predict(X_test_trans_1)

print("Accuracy: ", accuracy_score(y_test_1,reg_pred_1))

**Pattern 2 of training and validation.**

In [ ]:
imp_median = SimpleImputer(strategy='median')

# deleting the missing values
imp_X_1 = imp_median.fit_transform(X)

# One hot encoding
from sklearn.preprocessing import OneHotEncoder
enc_1 = OneHotEncoder(handle_unknown='ignore')
enc_imp_X_1 = enc.fit_transform(imp_X_1).toarray()

# splitting the data into training and testing data using train_test_split from sklearn
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(enc_imp_X_1, y, test_size=0.25, random_state=42)

# we standardized the data
scaler = StandardScaler()
scaler.fit(X_train_2)
X_train_trans_2 = scaler.transform(X_train_2)
X_test_trans_2 = scaler.transform(X_test_2)

# fitting the data
from lightgbm import LGBMClassifier
lgbm_1 = LGBMClassifier(random_state=5)
lgb_1 = lgbm_1.fit(X_train_trans_2, y_train_2)

# predicting
reg_pred_2 = lgb_1.predict(X_test_trans_2)

print("Accuracy: ", accuracy_score(y_test_2,reg_pred_2))

**Pattern 3 of training and validation.**

In [ ]:
imp_mf = SimpleImputer(strategy='most_frequent')

# deleting the missing values
imp_X_2 = imp_mf.fit_transform(X)

# One hot encoding
from sklearn.preprocessing import OneHotEncoder
enc_2 = OneHotEncoder(handle_unknown='ignore')
enc_imp_X_2 = enc.fit_transform(imp_X_2).toarray()

# splitting the data into training and testing data using train_test_split from sklearn
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(enc_imp_X_2, y, test_size=0.25, random_state=42)

# we standardized the data
scaler = StandardScaler()
scaler.fit(X_train_3)
X_train_trans_3 = scaler.transform(X_train_3)
X_test_trans_3 = scaler.transform(X_test_3)

# fitting the data
from lightgbm import LGBMClassifier
lgbm_2 = LGBMClassifier(random_state=5)
lgb_2 = lgbm_2.fit(X_train_trans_3, y_train_3)

# predicting
reg_pred_3 = lgb_2.predict(X_test_trans_3)

print("Accuracy: ", accuracy_score(y_test_3,reg_pred_3))

**Pattern 4 of training and validation.**

In [ ]:
imp_cnst = SimpleImputer(strategy='constant')

# deleting the missing values
imp_X_3 = imp_cnst.fit_transform(X)

# One hot encoding
from sklearn.preprocessing import OneHotEncoder
enc_3 = OneHotEncoder(handle_unknown='ignore')
enc_imp_X_3 = enc.fit_transform(imp_X_3).toarray()

# splitting the data into training and testing data using train_test_split from sklearn
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(enc_imp_X_3, y, test_size=0.25, random_state=42)

# we standardized the data
scaler = StandardScaler()
scaler.fit(X_train_4)
X_train_trans_4 = scaler.transform(X_train_4)
X_test_trans_4 = scaler.transform(X_test_4)

# fitting the data
from lightgbm import LGBMClassifier
lgbm_3 = LGBMClassifier(random_state=5)
lgb_3 = lgbm_3.fit(X_train_trans_4, y_train_4)

# predicting
reg_pred_4 = lgb_3.predict(X_test_trans_4)

print("Accuracy: ", accuracy_score(y_test_4,reg_pred_4))

**Pattern 5 of training and validation.**

In [ ]:
imp_cnst = SimpleImputer(strategy='constant')

# deleting the missing values
imp_X_4 = imp_cnst.fit_transform(X)

# One hot encoding
from sklearn.preprocessing import OneHotEncoder
enc_4 = OneHotEncoder(handle_unknown='ignore')
enc_imp_X_4 = enc.fit_transform(imp_X_4).toarray()

# splitting the data into training and testing data using train_test_split from sklearn
X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(enc_imp_X_4, y, test_size=0.25, random_state=42)

# we standardizied data
scaler = StandardScaler()
scaler.fit(X_train_5)
X_train_trans_5 = scaler.transform(X_train_5)
X_test_trans_5 = scaler.transform(X_test_5)

# fitting the data
from lightgbm import LGBMClassifier
lgbm_4 = LGBMClassifier(random_state=5)
lgb_4 = lgbm_4.fit(X_train_trans_5, y_train_5)

# predicting
reg_pred_5 = lgb_4.predict(X_test_trans_5)

print("Accuracy: ", accuracy_score(y_test_5,reg_pred_5))

* **The feature and preprocess used is** IMPUTAION and HOT ENCODING **techniques**.
* **These techniques are more useful, and from what is noticed, when using simple imputer the accuracy will still be high and they also stays constant.**